# 第三节 
 第三步骤下的模型 text-embedding-ada-002不可用。

以下是图片中出现的公式的LaTeX表示形式：

1. 能量信号与功率信号定义：
   $$
   E = \lim_{a \to \infty} \int_{-a}^{a} |f(t)|^2 \, dt, \quad P = \lim_{a \to \infty} \frac{1}{2a} \int_{-a}^{a} |f(t)|^2 \, dt
   $$

2. 指数函数：
   $$
   f(t) = Ke^{at}
   $$

3. 抽样函数：
   $$
   Sa(t) = \frac{\sin(\pi t)}{\pi t}, \quad Sa(t) = Sa(at), \quad \lim_{a \to 0} Sa(at) = 1, \quad \lim_{a \to \infty} Sa(at) = 0
   $$
   特性：
   $$
   t = k\pi, \, n = 1, 2, 3, 4, \ldots
   $$
   $$
   \int_{-\infty}^{\infty} Sa(t) \, dt = \pi, \quad \int_{0}^{\infty} Sa(t) \, dt = \frac{\pi}{2}
   $$

4. 门函数：
   $$
   f(t) = u\left(t + \frac{\tau}{2}\right) - u\left(t - \frac{\tau}{2}\right) = g_{\tau}(t)
   $$

5. 符号函数：
   $$
   sgn(t) = \begin{cases} 
      1 & t > 0 \\
      -1 & t < 0 
   \end{cases}
   $$
   或
   $$
   sgn(t) = -u(-t) + u(t) = 2u(t) - 1 = \frac{1}{2} \left[ sgn(t) + 1 \right]
   $$

6. 信号的基本运算与基本变换，包括加（减）、乘（除）、微分（积分）

In [ ]:
import os

os.environ$"OPENAI_BASE_URL"] = 'https://chatapi.littlewheat.com/v1'

# 1.Load 导入Document Loaders
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import TextLoader

# 加载Documents
base_dir = './OneFlower' # 文档的存放目录
documents = $]
for file in os.listdir(base_dir): 
    # 构建完整的文件路径
    file_path = os.path.join(base_dir, file)
    if file.endswith('.pdf'):
        loader = PyPDFLoader(file_path)
        documents.extend(loader.load())
    elif file.endswith('.docx'): 
        loader = Docx2txtLoader(file_path)
        documents.extend(loader.load())
    elif file.endswith('.txt'):
        loader = TextLoader(file_path)
        documents.extend(loader.load())


# 2.Split 将Documents切分成块以便后续进行嵌入和向量存储
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)
chunked_documents = text_splitter.split_documents(documents)



In [ ]:
"""
本文件是【用LangChain快速构建基于“易速鲜花”本地知识库的智能问答系统】章节的配套代码，课程链接：https://juejin.cn/book/7387702347436130304/section/7388069933021986856
您可以点击最上方的“运行“按钮，直接运行该文件；更多操作指引请参考Readme.md文件。
"""
# 1.Load 导入Document Loaders
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import TextLoader
from typing import Dict, List, Any
from langchain.embeddings.base import Embeddings
from langchain.pydantic_v1 import BaseModel
from volcenginesdkarkruntime import Ark

# os.environ$"OPENAI_API_KEY"] = '你的OpenAI API Key'
# os.environ$"OPENAI_BASE_URL"] = 'OpenAI 的 API URL'

# 加载Documents
base_dir = "./OneFlower"  # 文档的存放目录
documents = []
for file in os.listdir(base_dir):
    # 构建完整的文件路径
    file_path = os.path.join(base_dir, file)
    if file.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
        documents.extend(loader.load())
    elif file.endswith(".docx"):
        loader = Docx2txtLoader(file_path)
        documents.extend(loader.load())
    elif file.endswith(".txt"):
        loader = TextLoader(file_path)
        documents.extend(loader.load())

# 2.Split 将Documents切分成块以便后续进行嵌入和向量存储
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)
chunked_documents = text_splitter.split_documents(documents)


# 3.Store 将分割嵌入并存储在矢量数据库Qdrant中
from langchain_community.vectorstores import Qdrant


class DoubaoEmbeddings(BaseModel, Embeddings):
    client: Ark = None
    api_key: str = ""
    model: str

    def __init__(self, **data: Any):
        super().__init__(**data)
        if self.api_key == "":
            self.api_key = os.environ["OPENAI_API_KEY"]
        self.client = Ark(
            base_url=os.environ["OPENAI_BASE_URL"],
            api_key=self.api_key
        )

    def embed_query(self, text: str) -> List[float]:
        """
        生成输入文本的 embedding.
        Args:
            texts (str): 要生成 embedding 的文本.
        Return:
            embeddings (List[float]): 输入文本的 embedding，一个浮点数值列表.
        """
        embeddings = self.client.embeddings.create(model=self.model, input=text)
        return embeddings.data[0].embedding

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self.embed_query(text) for text in texts]

    class Config:
        arbitrary_types_allowed = True


vectorstore = Qdrant.from_documents(
    documents=chunked_documents,  # 以分块的文档
    embedding=DoubaoEmbeddings(
        model=os.environ["EMBEDDING_MODELEND"],
    ),  # 用OpenAI的Embedding Model做嵌入
    location=":memory:",  # in-memory 存储
    collection_name="my_documents",
)  # 指定collection_name

# 4. Retrieval 准备模型和Retrieval链
import logging  # 导入Logging工具
from langchain_openai import ChatOpenAI  # ChatOpenAI模型
from langchain.retrievers.multi_query import (
    MultiQueryRetriever,
)  # MultiQueryRetriever工具
from langchain.chains import RetrievalQA  # RetrievalQA链

# 设置Logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# 实例化一个大模型工具 - OpenAI的GPT-3.5
llm = ChatOpenAI(model=os.environ["LLM_MODELEND"], temperature=0)

# 实例化一个MultiQueryRetriever
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

# 实例化一个RetrievalQA链
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever_from_llm)

# 5. Output 问答系统的UI实现
from flask import Flask, request, render_template

app = Flask(__name__)  # Flask APP


@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        # 接收用户输入作为问题
        question = request.form.get("question")

        # RetrievalQA链 - 读入问题，生成答案
        result = qa_chain({"query": question})

        # 把大模型的回答结果返回网页进行渲染
        return render_template("index.html", result=result)

    return render_template("index.html")


if __name__ == "__main__":
    app.run(host="0.0.0.0", debug=True, port=5000)


ModuleNotFoundError: No module named 'volcenginesdkarkruntime'

In [ ]:


# 4. Retrieval 准备模型和Retrieval链
import logging # 导入Logging工具
from langchain.chat_models import ChatOpenAI # ChatOpenAI模型
from langchain.retrievers.multi_query import MultiQueryRetriever # MultiQueryRetriever工具
from langchain.chains import RetrievalQA # RetrievalQA链

# 设置Logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

# 实例化一个大模型工具 - OpenAI的GPT-3.5
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 实例化一个MultiQueryRetriever
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vectorstore.as_retriever(), llm=llm)

# 实例化一个RetrievalQA链
qa_chain = RetrievalQA.from_chain_type(llm,retriever=retriever_from_llm)

# 5. Output 问答系统的UI实现
from flask import Flask, request, render_template
app = Flask(__name__) # Flask APP

@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':

        # 接收用户输入作为问题
        question = request.form.get('question')

        # RetrievalQA链 - 读入问题，生成答案
        result = qa_chain({"query": question})
        # 把大模型的回答结果返回网页进行渲染
        return render_template('index.html', result=result)

    return render_template('index.html')

if __name__ == "__main__":
    app.run(host='0.0.0.0',debug=True,port=5000)

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

os.environ["OPENAI_BASE_URL"] = 'https://chatapi.littlewheat.com/v1'

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.8, max_tokens=600)

messages = [
    SystemMessage(content="你是一个很棒的智能助手"),
    HumanMessage(content="请给我的花店起个最好的名字"),
]
response = chat(messages)
print(response.content)




C:\Users\34435\AppData\Local\Temp\ipykernel_103860\1272913331.py:13: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = chat(messages)


当然可以！以下是一些富有创意的花店名字建议：

1. 花语花坊
2. 花香四溢
3. 梦幻花园
4. 幸福花季
5. 甜蜜花语
6. 生命之花
7. 爱之花巢
8. 花海情缘
9. 春意盎然
10. 心语花坊

希望这些名字能激发你的灵感！如果有特定的风格或主题，欢迎告诉我，我可以提供更符合你需求的建议。


# 2


In [4]:
import os
from openai import OpenAI

os.environ["OPENAI_BASE_URL"] = 'https://chatapi.littlewheat.com/v1'

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    #api_key='sk-tDyutqyLwfpf62lkHPgc4D01z7VWRF4Zm4voUQ4cRhMIt73v',      
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "请给我的花店起个名字",
        }
    ],
    temperature=0.8,
    max_tokens=60,
    model="gpt-3.5-turbo",)

print(chat_completion.choices[0].message.content)



当然可以！以下是一些花店名字的建议：

1. 花语芳香
2. 静谧花园
3. 花与心
4. 梦幻花坊
5. 四季花语
6. 绿意盎然
7. 花
